# Aprendizaje automático relacional

#### Fernando Jesús Fernández Gallardo
#### Carmen Galván López

## Preparación

#### Imports y variables globales

In [14]:
import pandas
import numpy
import sklearn
import networkx as nx
from sklearn import preprocessing, model_selection, naive_bayes
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

semilla = 86

#### Lectura y procesamiento inicial de los datos brutos

In [15]:
#Leemos los archivos
aristas = pandas.read_csv('data/political-books-nodes.csv')
vertices = pandas.read_csv('data/political-books-edges.csv')

#Borramos la columna ID
del(aristas['Id'])

#Mostramos las primeras 35 filas
aristas.head(35)

,Label,political_ideology
0,1000 Years for Revenge,neutral
1,Bush vs. the Beltway,conservative
2,Charlie Wilson's War,conservative
3,Losing Bin Laden,conservative
4,Sleeping With the Devil,neutral
5,The Man Who Warned America,conservative
6,Why America Slept,neutral
7,Ghost Wars,neutral
8,A National Party No More,conservative
9,Bush Country,conservative


#### Selección y validación de los datos brutos

In [16]:
"""
Comprobamos que el dataset es válido verificando que no existen duplicados
"""
if len(aristas) != len(set(aristas['Label'])):
    raise ValueError("El dataset no es válido ya que contiene duplicados")
"""
La mejor forma de identificar cada uno de los elementos que forma parte
del conjunto de entrenamiento es el nombre del propio libro (que en el dataset
se llama 'Label') en vez del ID o cualquier otro tipo de indentificador más
complejo. De esta forma, también es más fácil identificar elementos duplicados
(si los hubiera)
"""
atributos = aristas['Label']
"""
Nuestro objetivo es predecir la ideología política del autor basándonos en
sus obras, por lo que el objetivo que perseguimos en nuestro modelo
es el de la ideología política
"""
objetivos = aristas['political_ideology']

## Inicio del entrenamiento
#### Codificación del objetivo

## KNN no relacional

In [40]:
from sklearn.preprocessing import LabelEncoder

# Crea una instancia del codificador de etiquetas
label_encoder = LabelEncoder()

# Realiza la codificación de etiquetas en los atributos
atributos_encoded = label_encoder.fit_transform(atributo)

# Imprime los atributos codificados
print(atributos_encoded)

[  0  15  18  44  59  81 102  31   1  13  22  40  47  49  54  11   8  57
  45  89  65  36  77  64  34  92  27  61   3  20  87  38  73  96  83  41
  95   9  58  90  21  32  74  88 101  84  51   7  85  68  94  63   4 103
  37  29  97  76  52  24  62  55  66  72   5  70  79  53  60  19  91   2
   6  10  80 104  50  14  82  17  67  43  86  28  16  23  42  46  69  48
  35  75  30  33  39  98 100  71  99  25  93  56  12  78  26]


In [41]:
# Crea una instancia del codificador de etiquetas
label_encoder = LabelEncoder()

# Realiza la codificación de etiquetas en el objetivo
objetivo_encoded = label_encoder.fit_transform(objetivo)

# Imprime el objetivo codificado
print(objetivo_encoded)


[2 0 0 0 2 0 2 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 2 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 2 0 2 0 0 2 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1
 1 1 2 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2]


In [43]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

def encontrar_mejor_k(atributos, objetivo, k_range, cv=5):
    k_scores = []
    
    # Redimensionar los atributos a una matriz bidimensional
    atributos_2d = np.reshape(atributos, (-1, 1))
    
    for k in k_range:
        # Crear instancia del clasificador KNN con k vecinos
        knn = KNeighborsClassifier(n_neighbors=k)
        
        # Realizar validación cruzada
        scores = cross_val_score(knn, atributos_2d, objetivo, cv=cv)
        
        # Obtener el score promedio de la validación cruzada
        score_promedio = scores.mean()
        
        # Guardar el score promedio en la lista de puntajes
        k_scores.append(score_promedio)
        
    return k_scores

k_range = range(1, 11)  # Rango de valores de k que quieres probar
k_scores = encontrar_mejor_k(atributos_encoded, objetivo_encoded, k_range)

print("Puntajes para cada valor de k:", k_scores)


Puntajes para cada valor de k: [0.2761904761904762, 0.3333333333333333, 0.3142857142857143, 0.3142857142857142, 0.2952380952380952, 0.3333333333333333, 0.3142857142857143, 0.3238095238095238, 0.36190476190476184, 0.35238095238095235]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

In [17]:

"""
Para poder trabajar con los datos que tenemos, necesitamos convertirlos en un formato que sklearn pueda "entender".
Debemos de hacer que nuestros datos "planos" sean para sklearn objetos "comparables", dependiendo del tipo de
ordenación que nosotros veamos más apropiada para el método en cuestión
(de una manera similar hacemos en Java cuando implementamos la interfaz 'Comparable' y el método compareTo)

El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja
con una lista o array unidimensional de sus valores y admite cadenas

"""
# Codificadores
codificador_atributos = preprocessing.LabelEncoder()
codificador_objetivos = preprocessing.LabelEncoder()
# Datos codificados
atributos_codificados = codificador_atributos.fit_transform(atributos)
objetivos_codificados = codificador_objetivos.fit_transform(objetivos)

#### División en conjunto de entrenamiento y conjunto de prueba

Partimos el atributo y el objetivo en dos, de entrenamiento y de prueba

In [18]:
(atributos_entrenamiento,
 atributos_prueba,
 objetivos_entrenamiento,
 objetivos_prueba) = model_selection.train_test_split(
        atributos_codificados,
        objetivos_codificados,
        # Valor de la semilla aleatoria para que el muestreo sea reproducible a pesar de ser aleatorio
        random_state=semilla,
        test_size=.33,
        stratify=objetivos_codificados
)

## KNN no relacional

In [20]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

def encontrar_mejor_k(atributos, objetivo, k_range, cv=5):
    k_scores = []
    
    # Redimensionar los atributos a una matriz bidimensional
    atributos_2d = np.reshape(atributos, (-1, 1))
    
    for k in k_range:
        # Crear instancia del clasificador KNN con k vecinos
        knn = KNeighborsClassifier(n_neighbors=k)
        
        # Realizar validación cruzada
        scores = cross_val_score(knn, atributos_2d, objetivo, cv=cv)
        
        # Obtener el score promedio de la validación cruzada
        score_promedio = scores.mean()
        
        # Guardar el score promedio en la lista de puntajes
        k_scores.append(score_promedio)
        
    return k_scores

k_range = range(1, 11)  # Rango de valores de k que quieres probar
k_scores = encontrar_mejor_k(atributos_codificados, objetivos_codificados, k_range)

print("Puntajes para cada valor de k:", k_scores)


Puntajes para cada valor de k: [0.2761904761904762, 0.3333333333333333, 0.3142857142857143, 0.3142857142857142, 0.2952380952380952, 0.3333333333333333, 0.3142857142857143, 0.3238095238095238, 0.36190476190476184, 0.35238095238095235]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim